In [1]:
import pennylane as qml
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss

In [2]:
# Create a toy dataset
X, y = make_moons(n_samples=1000, noise=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Standardize the data
print(y_train.shape)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

(800,)


In [3]:
print(X_train)

[[ 1.76755973  0.05440598]
 [-1.76617114  0.31537521]
 [-0.72279926  1.65748645]
 ...
 [-1.46428304  0.67329721]
 [ 0.66197793 -0.22513462]
 [ 1.64081996 -0.23154454]]


In [4]:
n_qubits = 2  # Number of qubits
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(x, weights):
    # Angle embedding
    qml.templates.AngleEmbedding(x, wires=range(n_qubits))

    # Basic entangler layers
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    
    # Measurement
    return qml.expval(qml.PauliZ(0))


In [11]:
def cost(weights, X, y):
    # Combine X and y into a single array for easier processing
    combined = np.column_stack((X, y))
    
    # Use NumPy's vectorized operations
    predictions = circuit(combined[:, :-1], weights)
    predictions = (predictions + 1) / 2
    
    # Clip predictions to avoid log(0)
    epsilon = 1e-12
    predictions = np.clip(predictions, epsilon, 1 - epsilon)
    
    # Calculate Binary Cross-Entropy loss
    bce_loss = -np.mean(y * np.log(predictions) + (1 - y) * np.log(1 - predictions))
    
    return bce_loss
n_layers = 4
weights = np.random.randn(n_layers, 2, 3)  # (layers, rotations, qubits)

In [12]:
circuit_draw = qml.draw(circuit,level="device")(X_train[0], weights)  # Visualize with the first training sample
print(circuit_draw)


0: ──RX(1.77)──Rot(0.29,0.05,-0.69)─╭●─╭X──Rot(-0.29,1.22,0.06)──╭●─╭X──Rot(0.26,-0.54,-1.02)─╭●─╭X
1: ──RX(0.05)──Rot(-0.17,0.86,1.05)─╰X─╰●──Rot(-1.77,-0.37,0.53)─╰X─╰●──Rot(0.36,-0.69,-0.49)─╰X─╰●

───Rot(-0.61,0.38,1.08)──╭●─╭X─┤  <Z>
───Rot(-0.28,0.91,-0.95)─╰X─╰●─┤     


In [13]:
from scipy.optimize import minimize

# Optimize the weights
print(weights.flatten())
result = minimize(lambda w: cost(w.reshape(n_layers, 2, 3), X_train, y_train), 
                  weights.flatten(), method='Powell')
optimized_weights = result.x.reshape(n_layers, 2, 3)



[ 0.2928727   0.04555106 -0.69048381 -0.17484763  0.86333038  1.04650458
 -0.28851314  1.2192413   0.05563221 -1.76547252 -0.36546721  0.52612036
  0.26430212 -0.53575285 -1.0235261   0.36135714 -0.69464777 -0.49280703
 -0.60580837  0.37933657  1.07581176 -0.27682855  0.91370317 -0.95278777]


In [17]:
def predict(X):
    predictions = [circuit(x, optimized_weights) for x in X]
    predictions = np.array(predictions)
    return np.round((predictions + 1) / 2)  # Convert to binary (0 or 1)

# Make predictions
y_pred = predict(X_test)

In [18]:
accuracy_score(y_pred,y_test)

0.875